In [18]:
"""
This module imputes data using KNN imputation.
Author: Anuvrat Chaturvedi
Date: 2024-06-09
Inspired by: https://machinelearningmastery.com/knn-imputation-for-missing-values-in-machine-learning/
"""

'\nThis module imputes data using KNN imputation.\nAuthor: Anuvrat Chaturvedi\nDate: 2024-06-09\nInspired by: https://machinelearningmastery.com/knn-imputation-for-missing-values-in-machine-learning/\n'

### Import libraries and define standard variables

In [63]:
# Importing the required libraries
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, RepeatedStratifiedKFold
# IterativeImputer explicitly require this experimental feature
from sklearn.experimental import enable_iterative_imputer  # noqa
from sklearn.impute import KNNImputer, IterativeImputer, SimpleImputer
from sklearn.linear_model import BayesianRidge

In [51]:
# Define common variables
data_folder = '../data/'
seed=42

### Load the datasets

In [8]:
# Explore the pickled set of combined edgar data for the latest year and quarter
df_edgar_combined = pd.read_pickle(data_folder + 'df_edgar_combined.pkl')
df_edgar_combined.head()

,sym,IncomeTaxExpenseBenefit_CY2023,NetCashProvidedByUsedInFinancingActivities_CY2023,WeightedAverageNumberOfSharesOutstandingBasic_CY2023,WeightedAverageNumberOfDilutedSharesOutstanding_CY2023,NetCashProvidedByUsedInInvestingActivities_CY2023,NetCashProvidedByUsedInOperatingActivities_CY2023,CashCashEquivalentsRestrictedCashAndRestrictedCashEquivalentsPeriodIncreaseDecreaseIncludingExchangeRateEffect,ComprehensiveIncomeNetOfTax_CY2023,NetIncomeLoss_CY2023,...,Assets,LiabilitiesAndStockholdersEquity,AccumulatedOtherComprehensiveIncomeLossNetOfTax,RetainedEarningsAccumulatedDeficit,CashCashEquivalentsRestrictedCashAndRestrictedCashEquivalents,StockholdersEquity,Goodwill,AssetsCurrent,LiabilitiesCurrent,CashAndCashEquivalentsAtCarryingValue
0,A,9.900000e+07,-9.300000e+08,2.940000e+08,2.960000e+08,-3.100000e+08,1.772000e+09,5.370000e+08,1.260000e+09,1.240000e+09,...,1.085600e+10,1.085600e+10,-3.370000e+08,1.090000e+09,1.674000e+09,6.214000e+09,3.963000e+09,4.203000e+09,1.958000e+09,1.671000e+09
1,AAL,2.990000e+08,-3.206000e+09,6.536120e+08,7.196690e+08,-5.020000e+08,3.803000e+09,9.500000e+07,5.130000e+08,8.220000e+08,...,6.438400e+10,6.438400e+10,-4.877000e+09,-8.001000e+09,7.030000e+08,-5.500000e+09,4.091000e+09,1.454200e+10,2.499300e+10,NaN
2,AAPL,1.674100e+10,-1.084880e+11,1.574423e+10,1.581255e+10,3.705000e+09,1.105430e+11,5.760000e+09,9.665200e+10,9.699500e+10,...,3.374110e+11,3.374110e+11,-8.960000e+09,4.339000e+09,3.392100e+10,7.419400e+10,NaN,1.284160e+11,1.238220e+11,3.269500e+10
3,ABT,9.410000e+08,-7.091000e+09,1.740000e+09,1.749000e+09,-3.133000e+09,7.261000e+09,-2.986000e+09,5.935000e+09,5.723000e+09,...,7.246700e+10,7.246700e+10,-8.166000e+09,3.801100e+10,6.284000e+09,3.881000e+10,2.338300e+10,2.237600e+10,1.402100e+10,6.284000e+09
4,ACGL,-8.730000e+08,-6.900000e+07,3.687000e+08,3.788000e+08,-5.468000e+09,5.749000e+09,2.250000e+08,5.413000e+09,4.443000e+09,...,6.276800e+10,6.276800e+10,-8.210000e+08,2.140500e+10,1.600000e+09,1.935500e+10,NaN,NaN,NaN,9.930000e+08


In [33]:
# Keeping only numeric columns for imputation
df_edgar_combined_numeric = df_edgar_combined.select_dtypes(include='number')

### Simple Imputer

Three simple imputation methods will be used - constant(0), mean and median

In [67]:
# Define the imputer to be used
imputer = SimpleImputer(strategy='constant', fill_value=0, missing_values=np.nan, add_indicator=False)

# Fit the imputer on the data
imputer.fit(df_edgar_combined_numeric)

# Impute the data
df_edgar_combined_imputed = imputer.transform(df_edgar_combined_numeric)

# Fill rate of the imputed data
fill_rate_imputed = np.mean(~np.isnan(df_edgar_combined_imputed).flatten())
print(f"Missing: {sum(np.isnan(df_edgar_combined_imputed).flatten())}")

# Convert the imputed data to a dataframe
df_edgar_combined_imputed_simple_zero = pd.concat([df_edgar_combined.sym, pd.DataFrame(df_edgar_combined_imputed, columns=df_edgar_combined_numeric.columns)], axis=1)

# Display the imputed data
display(df_edgar_combined_imputed_simple_zero.head())

# Save the imputed data
df_edgar_combined_imputed_simple_zero.to_pickle(data_folder + 'df_edgar_combined_imputed_simple_zero.pkl')

Missing: 0


,sym,IncomeTaxExpenseBenefit_CY2023,NetCashProvidedByUsedInFinancingActivities_CY2023,WeightedAverageNumberOfSharesOutstandingBasic_CY2023,WeightedAverageNumberOfDilutedSharesOutstanding_CY2023,NetCashProvidedByUsedInInvestingActivities_CY2023,NetCashProvidedByUsedInOperatingActivities_CY2023,CashCashEquivalentsRestrictedCashAndRestrictedCashEquivalentsPeriodIncreaseDecreaseIncludingExchangeRateEffect,ComprehensiveIncomeNetOfTax_CY2023,NetIncomeLoss_CY2023,...,Assets,LiabilitiesAndStockholdersEquity,AccumulatedOtherComprehensiveIncomeLossNetOfTax,RetainedEarningsAccumulatedDeficit,CashCashEquivalentsRestrictedCashAndRestrictedCashEquivalents,StockholdersEquity,Goodwill,AssetsCurrent,LiabilitiesCurrent,CashAndCashEquivalentsAtCarryingValue
0,A,9.900000e+07,-9.300000e+08,2.940000e+08,2.960000e+08,-3.100000e+08,1.772000e+09,5.370000e+08,1.260000e+09,1.240000e+09,...,1.085600e+10,1.085600e+10,-3.370000e+08,1.090000e+09,1.674000e+09,6.214000e+09,3.963000e+09,4.203000e+09,1.958000e+09,1.671000e+09
1,AAL,2.990000e+08,-3.206000e+09,6.536120e+08,7.196690e+08,-5.020000e+08,3.803000e+09,9.500000e+07,5.130000e+08,8.220000e+08,...,6.438400e+10,6.438400e+10,-4.877000e+09,-8.001000e+09,7.030000e+08,-5.500000e+09,4.091000e+09,1.454200e+10,2.499300e+10,0.000000e+00
2,AAPL,1.674100e+10,-1.084880e+11,1.574423e+10,1.581255e+10,3.705000e+09,1.105430e+11,5.760000e+09,9.665200e+10,9.699500e+10,...,3.374110e+11,3.374110e+11,-8.960000e+09,4.339000e+09,3.392100e+10,7.419400e+10,0.000000e+00,1.284160e+11,1.238220e+11,3.269500e+10
3,ABT,9.410000e+08,-7.091000e+09,1.740000e+09,1.749000e+09,-3.133000e+09,7.261000e+09,-2.986000e+09,5.935000e+09,5.723000e+09,...,7.246700e+10,7.246700e+10,-8.166000e+09,3.801100e+10,6.284000e+09,3.881000e+10,2.338300e+10,2.237600e+10,1.402100e+10,6.284000e+09
4,ACGL,-8.730000e+08,-6.900000e+07,3.687000e+08,3.788000e+08,-5.468000e+09,5.749000e+09,2.250000e+08,5.413000e+09,4.443000e+09,...,6.276800e+10,6.276800e+10,-8.210000e+08,2.140500e+10,1.600000e+09,1.935500e+10,0.000000e+00,0.000000e+00,0.000000e+00,9.930000e+08


In [64]:
# Define the imputer to be used
imputer = SimpleImputer(strategy='mean', missing_values=np.nan, add_indicator=False)

# Fit the imputer on the data
imputer.fit(df_edgar_combined_numeric)

# Impute the data
df_edgar_combined_imputed = imputer.transform(df_edgar_combined_numeric)

# Fill rate of the imputed data
fill_rate_imputed = np.mean(~np.isnan(df_edgar_combined_imputed).flatten())
print(f"Missing: {sum(np.isnan(df_edgar_combined_imputed).flatten())}")

# Convert the imputed data to a dataframe
df_edgar_combined_imputed_simple_mean = pd.concat([df_edgar_combined.sym, pd.DataFrame(df_edgar_combined_imputed, columns=df_edgar_combined_numeric.columns)], axis=1)

# Display the imputed data
display(df_edgar_combined_imputed_simple_mean.head())

# Save the imputed data
df_edgar_combined_imputed_simple_mean.to_pickle(data_folder + 'df_edgar_combined_imputed_simple_mean.pkl')


Missing: 0


,sym,IncomeTaxExpenseBenefit_CY2023,NetCashProvidedByUsedInFinancingActivities_CY2023,WeightedAverageNumberOfSharesOutstandingBasic_CY2023,WeightedAverageNumberOfDilutedSharesOutstanding_CY2023,NetCashProvidedByUsedInInvestingActivities_CY2023,NetCashProvidedByUsedInOperatingActivities_CY2023,CashCashEquivalentsRestrictedCashAndRestrictedCashEquivalentsPeriodIncreaseDecreaseIncludingExchangeRateEffect,ComprehensiveIncomeNetOfTax_CY2023,NetIncomeLoss_CY2023,...,Assets,LiabilitiesAndStockholdersEquity,AccumulatedOtherComprehensiveIncomeLossNetOfTax,RetainedEarningsAccumulatedDeficit,CashCashEquivalentsRestrictedCashAndRestrictedCashEquivalents,StockholdersEquity,Goodwill,AssetsCurrent,LiabilitiesCurrent,CashAndCashEquivalentsAtCarryingValue
0,A,9.900000e+07,-9.300000e+08,2.940000e+08,2.960000e+08,-3.100000e+08,1.772000e+09,5.370000e+08,1.260000e+09,1.240000e+09,...,1.085600e+10,1.085600e+10,-3.370000e+08,1.090000e+09,1.674000e+09,6.214000e+09,3.963000e+09,4.203000e+09,1.958000e+09,1.671000e+09
1,AAL,2.990000e+08,-3.206000e+09,6.536120e+08,7.196690e+08,-5.020000e+08,3.803000e+09,9.500000e+07,5.130000e+08,8.220000e+08,...,6.438400e+10,6.438400e+10,-4.877000e+09,-8.001000e+09,7.030000e+08,-5.500000e+09,4.091000e+09,1.454200e+10,2.499300e+10,4.128997e+09
2,AAPL,1.674100e+10,-1.084880e+11,1.574423e+10,1.581255e+10,3.705000e+09,1.105430e+11,5.760000e+09,9.665200e+10,9.699500e+10,...,3.374110e+11,3.374110e+11,-8.960000e+09,4.339000e+09,3.392100e+10,7.419400e+10,9.693617e+09,1.284160e+11,1.238220e+11,3.269500e+10
3,ABT,9.410000e+08,-7.091000e+09,1.740000e+09,1.749000e+09,-3.133000e+09,7.261000e+09,-2.986000e+09,5.935000e+09,5.723000e+09,...,7.246700e+10,7.246700e+10,-8.166000e+09,3.801100e+10,6.284000e+09,3.881000e+10,2.338300e+10,2.237600e+10,1.402100e+10,6.284000e+09
4,ACGL,-8.730000e+08,-6.900000e+07,3.687000e+08,3.788000e+08,-5.468000e+09,5.749000e+09,2.250000e+08,5.413000e+09,4.443000e+09,...,6.276800e+10,6.276800e+10,-8.210000e+08,2.140500e+10,1.600000e+09,1.935500e+10,9.693617e+09,1.421319e+10,1.216856e+10,9.930000e+08


In [65]:
# Define the imputer to be used
imputer = SimpleImputer(strategy='median', missing_values=np.nan, add_indicator=False)

# Fit the imputer on the data
imputer.fit(df_edgar_combined_numeric)

# Impute the data
df_edgar_combined_imputed = imputer.transform(df_edgar_combined_numeric)

# Fill rate of the imputed data
fill_rate_imputed = np.mean(~np.isnan(df_edgar_combined_imputed).flatten())
print(f"Missing: {sum(np.isnan(df_edgar_combined_imputed).flatten())}")

# Convert the imputed data to a dataframe
df_edgar_combined_imputed_simple_median = pd.concat([df_edgar_combined.sym, pd.DataFrame(df_edgar_combined_imputed, columns=df_edgar_combined_numeric.columns)], axis=1)

# Display the imputed data
display(df_edgar_combined_imputed_simple_median.head())

# Save the imputed data
df_edgar_combined_imputed_simple_median.to_pickle(data_folder + 'df_edgar_combined_imputed_simple_median.pkl')

Missing: 0


,sym,IncomeTaxExpenseBenefit_CY2023,NetCashProvidedByUsedInFinancingActivities_CY2023,WeightedAverageNumberOfSharesOutstandingBasic_CY2023,WeightedAverageNumberOfDilutedSharesOutstanding_CY2023,NetCashProvidedByUsedInInvestingActivities_CY2023,NetCashProvidedByUsedInOperatingActivities_CY2023,CashCashEquivalentsRestrictedCashAndRestrictedCashEquivalentsPeriodIncreaseDecreaseIncludingExchangeRateEffect,ComprehensiveIncomeNetOfTax_CY2023,NetIncomeLoss_CY2023,...,Assets,LiabilitiesAndStockholdersEquity,AccumulatedOtherComprehensiveIncomeLossNetOfTax,RetainedEarningsAccumulatedDeficit,CashCashEquivalentsRestrictedCashAndRestrictedCashEquivalents,StockholdersEquity,Goodwill,AssetsCurrent,LiabilitiesCurrent,CashAndCashEquivalentsAtCarryingValue
0,A,9.900000e+07,-9.300000e+08,2.940000e+08,2.960000e+08,-3.100000e+08,1.772000e+09,5.370000e+08,1.260000e+09,1.240000e+09,...,1.085600e+10,1.085600e+10,-3.370000e+08,1.090000e+09,1.674000e+09,6.214000e+09,3.963000e+09,4.203000e+09,1.958000e+09,1.671000e+09
1,AAL,2.990000e+08,-3.206000e+09,6.536120e+08,7.196690e+08,-5.020000e+08,3.803000e+09,9.500000e+07,5.130000e+08,8.220000e+08,...,6.438400e+10,6.438400e+10,-4.877000e+09,-8.001000e+09,7.030000e+08,-5.500000e+09,4.091000e+09,1.454200e+10,2.499300e+10,1.100000e+09
2,AAPL,1.674100e+10,-1.084880e+11,1.574423e+10,1.581255e+10,3.705000e+09,1.105430e+11,5.760000e+09,9.665200e+10,9.699500e+10,...,3.374110e+11,3.374110e+11,-8.960000e+09,4.339000e+09,3.392100e+10,7.419400e+10,4.527000e+09,1.284160e+11,1.238220e+11,3.269500e+10
3,ABT,9.410000e+08,-7.091000e+09,1.740000e+09,1.749000e+09,-3.133000e+09,7.261000e+09,-2.986000e+09,5.935000e+09,5.723000e+09,...,7.246700e+10,7.246700e+10,-8.166000e+09,3.801100e+10,6.284000e+09,3.881000e+10,2.338300e+10,2.237600e+10,1.402100e+10,6.284000e+09
4,ACGL,-8.730000e+08,-6.900000e+07,3.687000e+08,3.788000e+08,-5.468000e+09,5.749000e+09,2.250000e+08,5.413000e+09,4.443000e+09,...,6.276800e+10,6.276800e+10,-8.210000e+08,2.140500e+10,1.600000e+09,1.935500e+10,4.527000e+09,5.822725e+09,4.534000e+09,9.930000e+08


### KNN Imputer

In [32]:
# Define the imputer to be used
imputer = KNNImputer(n_neighbors=5, weights='uniform', metric='nan_euclidean')

# Fit the imputer on the data
imputer.fit(df_edgar_combined_numeric)

# Impute the data
df_edgar_combined_imputed = imputer.transform(df_edgar_combined_numeric)

# Fill rate of the imputed data
fill_rate_imputed = np.mean(~np.isnan(df_edgar_combined_imputed).flatten())
print(f"Missing: {sum(np.isnan(df_edgar_combined_imputed).flatten())}")

# Convert the imputed data to a dataframe
df_edgar_combined_imputed_knn = pd.concat([df_edgar_combined.sym, pd.DataFrame(df_edgar_combined_imputed, columns=df_edgar_combined_numeric.columns)], axis=1)

# Display the imputed data
display(df_edgar_combined_imputed_knn.head())

# Save the imputed data
df_edgar_combined_imputed_knn.to_pickle(data_folder + 'df_edgar_combined_imputed_knn.pkl')


Missing: 0


,sym,IncomeTaxExpenseBenefit_CY2023,NetCashProvidedByUsedInFinancingActivities_CY2023,WeightedAverageNumberOfSharesOutstandingBasic_CY2023,WeightedAverageNumberOfDilutedSharesOutstanding_CY2023,NetCashProvidedByUsedInInvestingActivities_CY2023,NetCashProvidedByUsedInOperatingActivities_CY2023,CashCashEquivalentsRestrictedCashAndRestrictedCashEquivalentsPeriodIncreaseDecreaseIncludingExchangeRateEffect,ComprehensiveIncomeNetOfTax_CY2023,NetIncomeLoss_CY2023,...,Assets,LiabilitiesAndStockholdersEquity,AccumulatedOtherComprehensiveIncomeLossNetOfTax,RetainedEarningsAccumulatedDeficit,CashCashEquivalentsRestrictedCashAndRestrictedCashEquivalents,StockholdersEquity,Goodwill,AssetsCurrent,LiabilitiesCurrent,CashAndCashEquivalentsAtCarryingValue
0,A,9.900000e+07,-9.300000e+08,2.940000e+08,2.960000e+08,-3.100000e+08,1.772000e+09,5.370000e+08,1.260000e+09,1.240000e+09,...,1.085600e+10,1.085600e+10,-3.370000e+08,1.090000e+09,1.674000e+09,6.214000e+09,3.963000e+09,4.203000e+09,1.958000e+09,1.671000e+09
1,AAL,2.990000e+08,-3.206000e+09,6.536120e+08,7.196690e+08,-5.020000e+08,3.803000e+09,9.500000e+07,5.130000e+08,8.220000e+08,...,6.438400e+10,6.438400e+10,-4.877000e+09,-8.001000e+09,7.030000e+08,-5.500000e+09,4.091000e+09,1.454200e+10,2.499300e+10,3.873860e+09
2,AAPL,1.674100e+10,-1.084880e+11,1.574423e+10,1.581255e+10,3.705000e+09,1.105430e+11,5.760000e+09,9.665200e+10,9.699500e+10,...,3.374110e+11,3.374110e+11,-8.960000e+09,4.339000e+09,3.392100e+10,7.419400e+10,4.110222e+10,1.284160e+11,1.238220e+11,3.269500e+10
3,ABT,9.410000e+08,-7.091000e+09,1.740000e+09,1.749000e+09,-3.133000e+09,7.261000e+09,-2.986000e+09,5.935000e+09,5.723000e+09,...,7.246700e+10,7.246700e+10,-8.166000e+09,3.801100e+10,6.284000e+09,3.881000e+10,2.338300e+10,2.237600e+10,1.402100e+10,6.284000e+09
4,ACGL,-8.730000e+08,-6.900000e+07,3.687000e+08,3.788000e+08,-5.468000e+09,5.749000e+09,2.250000e+08,5.413000e+09,4.443000e+09,...,6.276800e+10,6.276800e+10,-8.210000e+08,2.140500e+10,1.600000e+09,1.935500e+10,9.063140e+09,1.625911e+10,1.323053e+10,9.930000e+08


### Iterative Imputer

In [61]:
# Define the imputer to be used
imputer = IterativeImputer(estimator=BayesianRidge(), n_nearest_features=5, initial_strategy='mean', max_iter=10, sample_posterior=True, tol=0.001,
                            missing_values=np.nan, imputation_order='ascending', verbose=1, random_state=seed)

# Fit the imputer on the data
imputer.fit(df_edgar_combined_numeric)

# Impute the data
df_edgar_combined_imputed = imputer.transform(df_edgar_combined_numeric)

# Fill rate of the imputed data
fill_rate_imputed = np.mean(~np.isnan(df_edgar_combined_imputed).flatten())
print(f"Missing: {sum(np.isnan(df_edgar_combined_imputed).flatten())}")

# Convert the imputed data to a dataframe
df_edgar_combined_imputed_iterative = pd.concat([df_edgar_combined.sym, pd.DataFrame(df_edgar_combined_imputed, columns=df_edgar_combined_numeric.columns)], axis=1)

# Display the imputed data
display(df_edgar_combined_imputed_iterative.head())

# Save the imputed data
df_edgar_combined_imputed_iterative.to_pickle(data_folder + 'df_edgar_combined_imputed_iterative.pkl')


[IterativeImputer] Completing matrix with shape (428, 42)
[IterativeImputer] Completing matrix with shape (428, 42)
Missing: 0


,sym,IncomeTaxExpenseBenefit_CY2023,NetCashProvidedByUsedInFinancingActivities_CY2023,WeightedAverageNumberOfSharesOutstandingBasic_CY2023,WeightedAverageNumberOfDilutedSharesOutstanding_CY2023,NetCashProvidedByUsedInInvestingActivities_CY2023,NetCashProvidedByUsedInOperatingActivities_CY2023,CashCashEquivalentsRestrictedCashAndRestrictedCashEquivalentsPeriodIncreaseDecreaseIncludingExchangeRateEffect,ComprehensiveIncomeNetOfTax_CY2023,NetIncomeLoss_CY2023,...,Assets,LiabilitiesAndStockholdersEquity,AccumulatedOtherComprehensiveIncomeLossNetOfTax,RetainedEarningsAccumulatedDeficit,CashCashEquivalentsRestrictedCashAndRestrictedCashEquivalents,StockholdersEquity,Goodwill,AssetsCurrent,LiabilitiesCurrent,CashAndCashEquivalentsAtCarryingValue
0,A,9.900000e+07,-9.300000e+08,2.940000e+08,2.960000e+08,-3.100000e+08,1.772000e+09,5.370000e+08,1.260000e+09,1.240000e+09,...,1.085600e+10,1.085600e+10,-3.370000e+08,1.090000e+09,1.674000e+09,6.214000e+09,3.963000e+09,4.203000e+09,1.958000e+09,1.671000e+09
1,AAL,2.990000e+08,-3.206000e+09,6.536120e+08,7.196690e+08,-5.020000e+08,3.803000e+09,9.500000e+07,5.130000e+08,8.220000e+08,...,6.438400e+10,6.438400e+10,-4.877000e+09,-8.001000e+09,7.030000e+08,-5.500000e+09,4.091000e+09,1.454200e+10,2.499300e+10,1.685393e+10
2,AAPL,1.674100e+10,-1.084880e+11,1.574423e+10,1.581255e+10,3.705000e+09,1.105430e+11,5.760000e+09,9.665200e+10,9.699500e+10,...,3.374110e+11,3.374110e+11,-8.960000e+09,4.339000e+09,3.392100e+10,7.419400e+10,2.021873e+10,1.284160e+11,1.238220e+11,3.269500e+10
3,ABT,9.410000e+08,-7.091000e+09,1.740000e+09,1.749000e+09,-3.133000e+09,7.261000e+09,-2.986000e+09,5.935000e+09,5.723000e+09,...,7.246700e+10,7.246700e+10,-8.166000e+09,3.801100e+10,6.284000e+09,3.881000e+10,2.338300e+10,2.237600e+10,1.402100e+10,6.284000e+09
4,ACGL,-8.730000e+08,-6.900000e+07,3.687000e+08,3.788000e+08,-5.468000e+09,5.749000e+09,2.250000e+08,5.413000e+09,4.443000e+09,...,6.276800e+10,6.276800e+10,-8.210000e+08,2.140500e+10,1.600000e+09,1.935500e+10,8.669049e+09,8.799913e+09,1.303526e+10,9.930000e+08


In [38]:
df_edgar_combined.head()

,sym,IncomeTaxExpenseBenefit_CY2023,NetCashProvidedByUsedInFinancingActivities_CY2023,WeightedAverageNumberOfSharesOutstandingBasic_CY2023,WeightedAverageNumberOfDilutedSharesOutstanding_CY2023,NetCashProvidedByUsedInInvestingActivities_CY2023,NetCashProvidedByUsedInOperatingActivities_CY2023,CashCashEquivalentsRestrictedCashAndRestrictedCashEquivalentsPeriodIncreaseDecreaseIncludingExchangeRateEffect,ComprehensiveIncomeNetOfTax_CY2023,NetIncomeLoss_CY2023,...,Assets,LiabilitiesAndStockholdersEquity,AccumulatedOtherComprehensiveIncomeLossNetOfTax,RetainedEarningsAccumulatedDeficit,CashCashEquivalentsRestrictedCashAndRestrictedCashEquivalents,StockholdersEquity,Goodwill,AssetsCurrent,LiabilitiesCurrent,CashAndCashEquivalentsAtCarryingValue
0,A,9.900000e+07,-9.300000e+08,2.940000e+08,2.960000e+08,-3.100000e+08,1.772000e+09,5.370000e+08,1.260000e+09,1.240000e+09,...,1.085600e+10,1.085600e+10,-3.370000e+08,1.090000e+09,1.674000e+09,6.214000e+09,3.963000e+09,4.203000e+09,1.958000e+09,1.671000e+09
1,AAL,2.990000e+08,-3.206000e+09,6.536120e+08,7.196690e+08,-5.020000e+08,3.803000e+09,9.500000e+07,5.130000e+08,8.220000e+08,...,6.438400e+10,6.438400e+10,-4.877000e+09,-8.001000e+09,7.030000e+08,-5.500000e+09,4.091000e+09,1.454200e+10,2.499300e+10,NaN
2,AAPL,1.674100e+10,-1.084880e+11,1.574423e+10,1.581255e+10,3.705000e+09,1.105430e+11,5.760000e+09,9.665200e+10,9.699500e+10,...,3.374110e+11,3.374110e+11,-8.960000e+09,4.339000e+09,3.392100e+10,7.419400e+10,NaN,1.284160e+11,1.238220e+11,3.269500e+10
3,ABT,9.410000e+08,-7.091000e+09,1.740000e+09,1.749000e+09,-3.133000e+09,7.261000e+09,-2.986000e+09,5.935000e+09,5.723000e+09,...,7.246700e+10,7.246700e+10,-8.166000e+09,3.801100e+10,6.284000e+09,3.881000e+10,2.338300e+10,2.237600e+10,1.402100e+10,6.284000e+09
4,ACGL,-8.730000e+08,-6.900000e+07,3.687000e+08,3.788000e+08,-5.468000e+09,5.749000e+09,2.250000e+08,5.413000e+09,4.443000e+09,...,6.276800e+10,6.276800e+10,-8.210000e+08,2.140500e+10,1.600000e+09,1.935500e+10,NaN,NaN,NaN,9.930000e+08


In [40]:
df_edgar_combined_imputed_knn.head()

,sym,IncomeTaxExpenseBenefit_CY2023,NetCashProvidedByUsedInFinancingActivities_CY2023,WeightedAverageNumberOfSharesOutstandingBasic_CY2023,WeightedAverageNumberOfDilutedSharesOutstanding_CY2023,NetCashProvidedByUsedInInvestingActivities_CY2023,NetCashProvidedByUsedInOperatingActivities_CY2023,CashCashEquivalentsRestrictedCashAndRestrictedCashEquivalentsPeriodIncreaseDecreaseIncludingExchangeRateEffect,ComprehensiveIncomeNetOfTax_CY2023,NetIncomeLoss_CY2023,...,Assets,LiabilitiesAndStockholdersEquity,AccumulatedOtherComprehensiveIncomeLossNetOfTax,RetainedEarningsAccumulatedDeficit,CashCashEquivalentsRestrictedCashAndRestrictedCashEquivalents,StockholdersEquity,Goodwill,AssetsCurrent,LiabilitiesCurrent,CashAndCashEquivalentsAtCarryingValue
0,A,9.900000e+07,-9.300000e+08,2.940000e+08,2.960000e+08,-3.100000e+08,1.772000e+09,5.370000e+08,1.260000e+09,1.240000e+09,...,1.085600e+10,1.085600e+10,-3.370000e+08,1.090000e+09,1.674000e+09,6.214000e+09,3.963000e+09,4.203000e+09,1.958000e+09,1.671000e+09
1,AAL,2.990000e+08,-3.206000e+09,6.536120e+08,7.196690e+08,-5.020000e+08,3.803000e+09,9.500000e+07,5.130000e+08,8.220000e+08,...,6.438400e+10,6.438400e+10,-4.877000e+09,-8.001000e+09,7.030000e+08,-5.500000e+09,4.091000e+09,1.454200e+10,2.499300e+10,3.873860e+09
2,AAPL,1.674100e+10,-1.084880e+11,1.574423e+10,1.581255e+10,3.705000e+09,1.105430e+11,5.760000e+09,9.665200e+10,9.699500e+10,...,3.374110e+11,3.374110e+11,-8.960000e+09,4.339000e+09,3.392100e+10,7.419400e+10,4.110222e+10,1.284160e+11,1.238220e+11,3.269500e+10
3,ABT,9.410000e+08,-7.091000e+09,1.740000e+09,1.749000e+09,-3.133000e+09,7.261000e+09,-2.986000e+09,5.935000e+09,5.723000e+09,...,7.246700e+10,7.246700e+10,-8.166000e+09,3.801100e+10,6.284000e+09,3.881000e+10,2.338300e+10,2.237600e+10,1.402100e+10,6.284000e+09
4,ACGL,-8.730000e+08,-6.900000e+07,3.687000e+08,3.788000e+08,-5.468000e+09,5.749000e+09,2.250000e+08,5.413000e+09,4.443000e+09,...,6.276800e+10,6.276800e+10,-8.210000e+08,2.140500e+10,1.600000e+09,1.935500e+10,9.063140e+09,1.625911e+10,1.323053e+10,9.930000e+08


In [31]:
# Evaluation of the imputer can be done by training a model on the imputed dataset and evaluating its performance. 
# We can compare it with other imputation methods like mean imputation, median imputation, etc.